<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 8: SQL DML** 
_Where SQL takes action._

## **Learning Objectives**
### **Theory / Be able to explain ...**
- AAA

### **Skills / Know how to ...**
- AAA


--------
## **LESSON 8 HIGHLIGHTS**

In [ ]:
#@title Run this cell if video does not appear
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/joDkI1ttf9o" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

---
## **BIG PICTURE: CRUD on ACID**
The general theme of this lesson is about transactional processing that puts, alters, and deletes data in databases. In past lessons we discussed the four basic CRUD actions in the abstract. Now we will get into the nitty gritty details, or at least the ones that can be addressed with SQL.

In principle we'd like the data in our database to survive IT armageddon where the power shuts down with no notice, the database is in the middle of a lengthy operation, and the consequences of failure are catastrophic. Then perhaps we can begin to count on it being there when we need it. 

The gold standard for robustness in the face of catastrophic failure is ACID, four properties that together go as far as possible to keep our data safe:
- **Atomicity.** Lengthy transactions with lots of steps are treated as one unit. If any step fails then we can roll it back to the beginning, as if it never happened. 
- **Consistency.** We never want the database to be in an unexpected or unrecoverable state. If given data operations in any given order the result is always the same. There is no ambiguity or uncertainty introduced by the system itself. 
- **Isolation.** Just like people, databases often have to multitask, processing several transactions at once. Ideally, we want to keep failure of any transaction from causing failure of another. They should be running as independently as possible, *especially* when failure risk is high. 
- **Durability.** Once data has been committed by a transaction, it should persist until another transaction alters it. 

If you think about each of these things, you will realize just how fragile most software really is. 
- How far back in time does the "undo" on your word processor allow you to go back? If everything you have written since yesterday afternoon was garbage, could you recover it *even if you never saved it anywhere?* 
- If your computer crashed while it was halfway through saving the latest draft of your senior thesis, would the file be recoverable? Would you lose half your work?
- If you and a classmate are editing the same Google doc and your partner falls asleep at the keyboard, typing an infinite string of J characters ..., can you regain control before everything is destroyed? Or do you have to start over with a new doc? 
- If you start recording a workout on your smart watch but then forget to end the workout before all of the power is drained from your battery, does any data get lost? 

ACID is how we prevent all of these things from blowing up your data. 

Okay, so where do you get ACID? From any modern relational database product. Even the most primitive ones like SQLite or MS Access are better than any other tool for keeping your data safe. They are CRUD on ACID, and that's a good thing. 


---
## **ETL = Extract $\rightarrow$ Transform $\rightarrow$ Load**

While it is certainly wonderful and make the analyst's life much easier if data was collected expressly for their use, the typical case is not so great. Data can come from anywhere and may require significant scrubbing before it can be trusted. In some cases, there may be multiple data sources, with somewhat incompatible data to be merged into a coherent dataset. 

The general process of working with such *dirty* data is called ETL:
- **Extract** data from the original sources.
- **Transform** and integrate it to fit the new purpose.
- **Load it** into a central data repository that will protect the data from corruption.

While there are certainly other tools for this purpose, SQL is a great place to start:
- Modern relational databases include utilities for working with data in various formats. 
- SQL includes plenty of functions for transforming data from one data type to another *plus* the power of SQL queries to bring it all together into a useful form. **If SQL is not not enough, then use another tool as well.** SQL is already compatible with just about every programming language on earth. It has serious first mover advantage from decades of heavy use.
- When you are done, the data can reside safely in a database with the guarantees of full ACID compliance. 

After reviewing the syntax and function of SQL `INSERT`, `UPDATE`, and `DELETE` statements, we will consider a few special cases that put ACID principles to the test.  

 


---
## **SQL `INSERT` Statements** 
We use `INSERT` statements to add rows to a table. There are two basic forms:
- Adding new data (values) to the database
- Addind table data extracted (selected) from another table

### **`INSERT INTO ... VALUES`**

```sql
INSERT INTO tablename ( columnlist ) VALUES
  ( valuelist ); 
```

- `columnlist` and `valuelist` are comma-separated lists of column names and literal values. The two lists have exactly the same number of items, with the first column corresponding to the first value, etc.
- Any columns not included in the `columnlist` are not assigned a NULL value unless a `DEFAULT` value is specified.
- If the table has a surrogate primary key, then generally we do not want to include the primary key column; the database will generate it for us.
- When we say *literal values* in the `valuelist` we mean the values they would appear in a `WHERE` clause. It would be the value to the right of the `=` in a boolean expression. 
- The parentheses and trailing semicolon are not optional. 

We can insert multiple rows at a time as follows. 

```sql
INSERT INTO tablename ( columnlist ) VALUES
  ( valuelist1 ),
  ( valuelist2 ), 
  ...
  ;
```

- That's a list of `valuelist` items, one per row.
- There is no comma just before the semicolon.

For example, the following adds two new movies to the Movies Tonight database:
```sql
INSERT INTO movies ( title, rating ) VALUES 
  ('Romeo and Juliet', 'PG-13'),
  ('A Time to Kill','PG-13');
```
Note that the `movieID` column was omitted because it is autogenerated by the database. 


### `INSERT INTO ... SELECT`
If the data is already in the database in some form, then we can use a `SELECT` query to  gather the data values prior to insertion.

```sql
INSERT INTO tablename ( columnlist ) 
  SELECT ...
  ;
```

- As with inserting literal values the columns returned by the `SELECT` query must correspond to the ones in the `columnlist`
- The actual names of the columns returned by the `SELECT` query do not matter, though the data types should be compatible with what is already defined in the table. 

Another movies example, this time using data [imported from IMDB](https://www.imdb.com/interfaces/):

```sql
INSERT INTO movies ( title )  
  SELECT primaryTitle 
  FROM imdb_title_basics_import 
  WHERE startYear = "1996" 
```

- Since IMDB does not provide US movie ratings in its public data dumps the `ratings` column was omitted from the `columnlist`. That also means that the `ratings` column has to allow null values. Otherwise the insertion will fail. 
- If we want to keep track of the `tconst` movie identifier used by IMDB then we will have to add another column to the `movies` table. 



---
## **SQL `UPDATE` Statements**

SQL `UPDATE` statements set specific column values on selected rows. 

```sql
UPDATE tablename 
SET
  column1 = newvalue1,
  column2 = newvalue2,
  ...
WHERE ...
;
```
- Only the columns that are being updated need to be included.
- The `WHERE` clause works just like in a `SELECT` query.
- A new value can be any expression that returns a scalar value. That includes subqueries. 
- It is possible to use joins to update several tables at once. However, that is fairly new to the SQL standards and not likely to work in older (legacy) databases. It won't work n MySQL 5.7, for example, but it does work in MySQL 8.0. The workaround is to use subqueries (with joins) instead. 

Here we are updating the *Romeo + Juliet* movie title to its proper name.

```sql 
UPDATE movies 
SET
  title = 'Romeo + Juliet' 
WHERE
  movieID = 24;
```

---
## **SQL `DELETE` Statements**

Deleting rows is about as easy as it gets. 

```sql
DELETE FROM tablename
WHERE ...
```

- There is no need to specify columns.
- If the `WHERE` clause is omitted then *every* row is deleted. 
- We can delete from multiple tables at a time with slightly altered syntax. However, it is not universally supported. 














---
## **Load Order and Transactions**
Maintaining referential integrity is a continual process. The DBMS is always on the lookout for integrity violations. Each query is treated as an *atomic transaction* that can be undone (rolled back) if it does not complete sucessfully. Thus, is an update query sets a foreign key to an impossible value or nullified something that can't be null, then the database will immediately complain and return the database state to whatever it was before the query. 

While that is a very reasonable and safe way to approach data integrity, it has some implciations for how and when we load data into a given table. We will first consider cases where **table load order** can be used to avoid referential integrity violations. Then we will consider cases where we have to go further, using custom transaction controls to force the database to do what we need it to. 

### **Strongest First Loading**
The vast majority of referential integrity problems can be prevented by taking care about the order in which we insert and delete records. 

Consider any parent-child relationship where the parent must exist before the child. 

The process to add a new child row is then:
1. If the parent doesn't exist then add the parent first.
2. Once the parent exists, then add the child. 

Deleting the parent can cause the opposite problem, as all children will need to be deleted before the parent. We can use `ON DELETE CASCADE` in the foreign key constraints to handle that automatically.

When applied to a whole database the load order is strongest to weakest:
1. Load all the strong entities.
2. Load all weak entities that only depend on strng entities.
3. Load any entities that only rely on #1 and #2.
4. ...

We will see this strategy in place with the Movies Tonight case a little further down. 

### **Transaction Control**

Sometimes just taking care with load order is not enough. For those cases we use transaction control. 

Consider the classic parent-child-grandchild case, where there is a whole hierarchy of entities to be saved at once. This might happen, for example, when saving a new customer, the customer's order invoice, and the invoice line items. Based on the Strongest to Weakest rule, we would save the parent (customer record), then the child (invoice), and then the grand children (line items). 

The SQL code might look something like this:

```sql
INSERT INTO customers ...
INSERT INTO invoices ...  -- MySQL: use LAST_INSERT_ID() function to get the new customer id 
                          -- SQLite: use LAST_INSERT_ROWID()
INSERT INTO invoice_items ...
```

However, what happens if there is a problem saving one of the grand children? Then the entire transaction should be voided, including the invoice and new customer record. Instead of deleting them one by one, we can use a transaction block instead:

```sql
 
BEGIN;        -- Start a new Transaction
INSERT INTO customers ...  
INSERT INTO invoices ...
INSERT INTO invoice_items ...
COMMIT;       -- Finalize the Transaction
```

If the transaction fails before the `COMMIT` statement, then all changes made during the transaction are ignored. It's like it never happened. 

#### **Solving the Twinning Problem**
Transaction control can be used to solve the mandatory twins issue from Lesson 6. Depending on the database vendor, there are two different approaches. 

The most common (and theoretically correct) approach is **deferred commit**, where specific foreign key checks are marked as *deferrable* if they are inside of a transaction block. The keys are then checked as part of the `COMMIT` at the end. [This is how it works in SQLite](https://www.sqlite.org/foreignkeys.html), for example. 

A more risky approach is to **explictly disable foreign key checks** during a given transaction block. In MySQL this looks like:

```sql 
BEGIN;
SET foreign_key_checks = 0; -- disables FK checking
...
SET foreign_key_checks = 1; -- reenables FK checking
COMMIT;
```

The risk here is that one might forget to the `SET foreign_key_checks = 1;` step just before the `COMMIT`. 

---
## **Movies Tonight, Part 4**

We will finish off the Movies Tonight case by extracting, transforming, and loading the data from a single CSV file.  

![ERD from Lesson 5](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L6_MoviesTonight_v2.png)

### **Setup (Again)**

The code below creates a folder in Google Drive for our SQLite database. 











In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create the BUAN6510/data/MoviesTonight folder in Google Drive
from pathlib import Path
data_root = Path("./drive/My Drive/Colab Notebooks/BUAN6510")
if not data_root.exists():
  print(
      '''
      Warning! The folder '/Colab Notebooks/BUAN6510' could not be found in the connected Google Drive. 
      Please make 100% sure that both Colab and Chrome are set up use your @student.fairfield.edu account. 
      For now, a new folder with the correct path has been created in whatever Google Drive it found. 
      ''')
data_root = data_root / 'data' / 'MoviesTonight'
data_root.mkdir(parents=True, exist_ok=True)

This cell creates a symlink so that SQLite can find the new Google Drive folder. 

In [ ]:
%%bash
ln -s drive/My\ Drive/Colab\ Notebooks/BUAN6510 buan6510

The Google Drive ready to hold our data, we now move on to setting up %%sql magic and SQLite. While we are at it, we will create the new database file. 

In [ ]:
# Load %%sql magic
%load_ext sql

# Standard Imports
import sqlite3
import pandas as pd

# Database connection
%sql sqlite:///buan6510/data/MoviesTonight/MoviesTonight.db

### **Importing from CSV**


#### **Converting the Time Strings**

### **Populating the Strong Entities**

### **Populating the Weak Entities**

### **Kicking the Tires with a Few Queries**

### **A little cleanup before we go**

---
## **PRO TIPS: How to Create a Reliable and Repeatable ETL Process**



---
## **SQL AND BEYOND: FastAPI**













 







  

 








---
## **Congratulations! You've made it to the end of Lesson 8.**

It's all downhill from here. Just be sure to study for Quiz 4. 



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `BUAN6510` folder so you can find it next time.